In [ ]:
# Task 2: Time Series Forecasting Models

# Importing necessary libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from pmdarima import auto_arima

# Step 1: Load the data
# Fetch historical data for TSLA from YFinance (replace file paths with actual data sources)
tsla_data = pd.read_csv("C:/Users/user/Desktop/10 Academy- Machine-Learning/10 Academy W11/GMF_Investments_Project/data/TSLA.csv")
tsla_data['Date'] = pd.to_datetime(tsla_data['Date'])
tsla_data.set_index('Date', inplace=True)

# Visualize the stock data
tsla_data['Close'].plot(figsize=(10, 6))
plt.title('Tesla Stock Price Over Time')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.show()

# Step 2: Split the data into training and test sets
train_size = int(len(tsla_data) * 0.8)
train_data, test_data = tsla_data['Close'][:train_size], tsla_data['Close'][train_size:]

# Step 3: Apply ARIMA model (AutoRegressive Integrated Moving Average)
# ARIMA model can be used when the data shows no seasonal pattern (or minimal seasonality)
model_arima = ARIMA(train_data, order=(5, 1, 0))  # (p, d, q)
model_arima_fit = model_arima.fit()

# Forecasting the next 30 days
forecast_arima = model_arima_fit.forecast(steps=30)
forecast_arima = pd.Series(forecast_arima, index=test_data.index[:30])

# Plot the forecasted values vs actual values
plt.figure(figsize=(10, 6))
plt.plot(test_data[:30], label='Actual Prices', color='blue')
plt.plot(forecast_arima, label='ARIMA Forecasted Prices', color='red')
plt.title('ARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Step 4: Evaluate the ARIMA model
mae_arima = mean_absolute_error(test_data[:30], forecast_arima)
rmse_arima = np.sqrt(mean_squared_error(test_data[:30], forecast_arima))
mape_arima = mean_absolute_percentage_error(test_data[:30], forecast_arima)
print(f'ARIMA Model Evaluation:\nMAE: {mae_arima}\nRMSE: {rmse_arima}\nMAPE: {mape_arima}')

# Step 5: Apply SARIMA model (Seasonal ARIMA)
# SARIMA is used when the data has clear seasonal patterns
model_sarima = SARIMAX(train_data, order=(5, 1, 0), seasonal_order=(1, 1, 0, 5))  # (p, d, q, P, D, Q, m)
model_sarima_fit = model_sarima.fit()

# Forecasting the next 30 days with SARIMA
forecast_sarima = model_sarima_fit.forecast(steps=30)
forecast_sarima = pd.Series(forecast_sarima, index=test_data.index[:30])

# Plot the forecasted values vs actual values
plt.figure(figsize=(10, 6))
plt.plot(test_data[:30], label='Actual Prices', color='blue')
plt.plot(forecast_sarima, label='SARIMA Forecasted Prices', color='green')
plt.title('SARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Step 6: Evaluate the SARIMA model
mae_sarima = mean_absolute_error(test_data[:30], forecast_sarima)
rmse_sarima = np.sqrt(mean_squared_error(test_data[:30], forecast_sarima))
mape_sarima = mean_absolute_percentage_error(test_data[:30], forecast_sarima)
print(f'SARIMA Model Evaluation:\nMAE: {mae_sarima}\nRMSE: {rmse_sarima}\nMAPE: {mape_sarima}')

# Step 7: Implement LSTM model (for deep learning-based forecasting)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(tsla_data['Close'].values.reshape(-1, 1))

# Prepare the dataset for LSTM (using 60 days of previous data to predict the next day)
def prepare_data(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = prepare_data(scaled_data[:train_size])
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Build the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(units=50, return_sequences=False))
model_lstm.add(Dense(units=1))
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
model_lstm.fit(X_train, y_train, epochs=10, batch_size=32)

# Prepare the test data
scaled_test_data = scaler.transform(tsla_data['Close'][train_size:].values.reshape(-1, 1))
X_test, y_test = prepare_data(scaled_test_data)

# Predict the stock prices using LSTM
predicted_stock_price = model_lstm.predict(X_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

# Plot the actual vs predicted prices
plt.figure(figsize=(10, 6))
plt.plot(tsla_data['Close'][train_size:].index, tsla_data['Close'][train_size:], color='blue', label='Actual Prices')
plt.plot(tsla_data['Close'][train_size:].index, predicted_stock_price, color='red', label='LSTM Predicted Prices')
plt.title('LSTM Model Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Step 8: Evaluate the LSTM model
mae_lstm = mean_absolute_error(tsla_data['Close'][train_size:].values[-len(predicted_stock_price):], predicted_stock_price)
rmse_lstm = np.sqrt(mean_squared_error(tsla_data['Close'][train_size:].values[-len(predicted_stock_price):], predicted_stock_price))
mape_lstm = mean_absolute_percentage_error(tsla_data['Close'][train_size:].values[-len(predicted_stock_price):], predicted_stock_price)
print(f'LSTM Model Evaluation:\nMAE: {mae_lstm}\nRMSE: {rmse_lstm}\nMAPE: {mape_lstm}')
